In [248]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [404]:
def create_tfidf_features(corpus, max_features=2000000, max_df=0.97, min_df=1):
    
    tfidf_vectorizor = TfidfVectorizer(decode_error='replace', strip_accents='unicode', analyzer='word',
                                       ngram_range=(1, 3), max_features=max_features,
                                       norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,
                                       max_df=max_df, min_df=min_df)
    X = tfidf_vectorizor.fit_transform(corpus)
    return X, tfidf_vectorizor

def calculate_similarity(X, vectorizor, query, top_k=5):    
    query_vec = vectorizor.transform(query)
    cosine_similarities = cosine_similarity(X,query_vec).flatten()    
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

def show_similar_documents(df, cosine_similarities, similar_doc_indices):
    counter = 1
    for index in similar_doc_indices:
        if cosine_similarities[index] > 0.60:
            print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
            print('Hadees: {}'.format(index))
            print(df['Text'][index])
            counter += 1

In [405]:
data = pd.read_csv('data_of_ahadees.csv')
data = data[~data['Text'].isnull()].reset_index(drop=True)
data['Text'] = data['Text'].apply(lambda x: str(x).strip())

In [406]:
X,v = create_tfidf_features(data['Text'])

In [421]:
import pickle
# pickle.dump(X, open('tfidfFeaturesX.pkl','wb'))
# pickle.dump(v, open('tfidfVectorsV.pkl','wb'))
X = pickle.load(open('tfidfFeaturesX.pkl','rb'))
v = pickle.load(open('tfidfVectorsV.pkl','rb'))

In [422]:
features = v.get_feature_names()
len(features)

1276589

In [423]:
import string
user_question = [""]
sim_vecs, cosine_similarities = calculate_similarity(X, v, user_question,top_k=1)
show_similar_documents(data,cosine_similarities, sim_vecs)